# Read / Write Operations to MSSQL using Spark with Scala
___

### Importing Packages

In [1]:
import org.apache.spark.sql.{SparkSession,SaveMode}

### Adding MSSQL Dependancy Jar

In [2]:
%AddJar https://repo1.maven.org/maven2/com/microsoft/sqlserver/mssql-jdbc/7.2.2.jre8/mssql-jdbc-7.2.2.jre8.jar

Starting download from https://repo1.maven.org/maven2/com/microsoft/sqlserver/mssql-jdbc/7.2.2.jre8/mssql-jdbc-7.2.2.jre8.jar
Finished download of mssql-jdbc-7.2.2.jre8.jar


### Creating Spark Session

In [3]:
val spark = SparkSession.builder.appName("mssql").getOrCreate()

spark = org.apache.spark.sql.SparkSession@3829e6d5


org.apache.spark.sql.SparkSession@3829e6d5

### MSSQL Server Credentials

In [4]:
val driver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"
val url = "jdbc:sqlserver://192.168.2.19:1433"
val dbtable = "CLOUDEDH.dbo.SLI_ATTRIBUTES"
val user = "SA"
val password = "Aline-12#"

driver = com.microsoft.sqlserver.jdbc.SQLServerDriver
url = jdbc:sqlserver://192.168.2.19:1433
dbtable = CLOUDEDH.dbo.SLI_ATTRIBUTES
user = SA
password = Aline-12#


Aline-12#

### Reading File

In [6]:
var filedf = spark.read.format("csv")
                   .options(Map(("header","true"),("inferSchema","true"),("delimiter","\t")))
                   .load("../Resources/SI_attr.tab")

filedf = [Sales_Item_Id: string, Sales_Item_Description: string ... 4 more fields]


[Sales_Item_Id: string, Sales_Item_Description: string ... 4 more fields]

### Show

In [7]:
filedf.show(5)

+------------------+----------------------+-----+-----------------------+----------------+--------------------------------+
|     Sales_Item_Id|Sales_Item_Description|  GIC|Product_Lifecycle_State|Purchasing_Group|Average Standard Production Cost|
+------------------+----------------------+-----+-----------------------+----------------+--------------------------------+
|    IE2:15HP-RFU-7|           7GHZ HP ODU|07599|               OBSOLETE|            null|                             0.0|
|    IE2:15HP-RFU-8|  1500HP RF UNIT, F...|07599|               OBSOLETE|            null|                             0.0|
|IE2:15HP-SHORT-112|    1500HP SHORT, FGHZ|07599|               OBSOLETE|            null|                             0.0|
|IE2:15HP-SHORT-137|     1500P SHORT, FGHZ|07599|               OBSOLETE|            null|                             0.0|
| IE2:15HP-TERM-112|  1500HP 50 OHM TER...|09208|               OBSOLETE|            null|                             0.0|
+-------

### Schema

In [8]:
filedf.printSchema

root
 |-- Sales_Item_Id: string (nullable = true)
 |-- Sales_Item_Description: string (nullable = true)
 |-- GIC: string (nullable = true)
 |-- Product_Lifecycle_State: string (nullable = true)
 |-- Purchasing_Group: string (nullable = true)
 |-- Average Standard Production Cost: double (nullable = true)



### Reading table from MSSQL

In [9]:
val tableDF = spark.read
    .format("jdbc")
    .option("url", url)
    .option("dbtable", dbtable)
    .option("user", user)
    .option("password", password)
    .option("driver", driver)
    .load()

tableDF = [ID: bigint, CREATED_USER: string ... 9 more fields]


[ID: bigint, CREATED_USER: string ... 9 more fields]

### Checking Schema of the table

In [10]:
tableDF.printSchema

root
 |-- ID: long (nullable = true)
 |-- CREATED_USER: string (nullable = true)
 |-- MODIFIED_USER: string (nullable = true)
 |-- CREATED_DATE: timestamp (nullable = true)
 |-- MODIFIED_DATE: timestamp (nullable = true)
 |-- SALES_ITEM_DESCRIPTION: string (nullable = true)
 |-- PURCHASING_GROUP: string (nullable = true)
 |-- PRODUCT_LIFECYCLE_STATE: string (nullable = true)
 |-- GIC: string (nullable = true)
 |-- AVERAGE_STANDARD_PRODUCTION_COST: double (nullable = true)
 |-- SALES_ITEM_ID: string (nullable = true)



### Changing Dataframe columns same as table columns

In [11]:
filedf = filedf.withColumnRenamed("Average Standard Production Cost","AVERAGE_STANDARD_PRODUCTION_COST")

filedf = [Sales_Item_Id: string, Sales_Item_Description: string ... 4 more fields]


[Sales_Item_Id: string, Sales_Item_Description: string ... 4 more fields]

### Converting all file dataframe columns to lower case

In [12]:
filedf = filedf.toDF(filedf.columns map (_.toUpperCase): _*)

filedf = [SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 4 more fields]


[SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 4 more fields]

### Generating sequential id column using Window Functions

In [13]:
import org.apache.spark.sql.functions.lit
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._

In [14]:
filedf = filedf.withColumn("ID", lit(1))

filedf = [SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 5 more fields]


[SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 5 more fields]

In [15]:
var w = Window.orderBy("ID")

w = org.apache.spark.sql.expressions.WindowSpec@55be3327


org.apache.spark.sql.expressions.WindowSpec@55be3327

In [17]:
filedf = filedf.withColumn("ID",row_number.over(w))

filedf = [SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 5 more fields]


[SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 5 more fields]

### Creating Remaining Columns and adding default values

In [18]:
filedf = filedf.withColumn("CREATED_DATE", current_timestamp())
               .withColumn("CREATED_USER", lit("Varun CK"))
               .withColumn("MODIFIED_DATE", current_timestamp())
               .withColumn("MODIFIED_USER", lit("Varun CK"))

filedf = [SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 9 more fields]


[SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 9 more fields]

### Show

In [19]:
filedf.show(5)

+------------------+----------------------+-----+-----------------------+----------------+--------------------------------+---+--------------------+------------+--------------------+-------------+
|     SALES_ITEM_ID|SALES_ITEM_DESCRIPTION|  GIC|PRODUCT_LIFECYCLE_STATE|PURCHASING_GROUP|AVERAGE_STANDARD_PRODUCTION_COST| ID|        CREATED_DATE|CREATED_USER|       MODIFIED_DATE|MODIFIED_USER|
+------------------+----------------------+-----+-----------------------+----------------+--------------------------------+---+--------------------+------------+--------------------+-------------+
|    IE2:15HP-RFU-7|           7GHZ HP ODU|07599|               OBSOLETE|            null|                             0.0|  1|2020-02-12 13:28:...|    Varun CK|2020-02-12 13:28:...|     Varun CK|
|    IE2:15HP-RFU-8|  1500HP RF UNIT, F...|07599|               OBSOLETE|            null|                             0.0|  2|2020-02-12 13:28:...|    Varun CK|2020-02-12 13:28:...|     Varun CK|
|IE2:15HP-SHORT

### Checking Table Count before writing

In [20]:
tableDF.count()

0

### Writing to MSSQL

In [21]:
filedf.write
  .format("jdbc")
  .options(Map(("url", url),("dbtable", dbtable),("user", user),("password", password),("driver", driver)))
  .mode(SaveMode.Append)
  .save()

### Checking Table Count after writing

In [22]:
val tableDF = spark.read
    .format("jdbc")
    .option("url", url)
    .option("dbtable", dbtable)
    .option("user", user)
    .option("password", password)
    .option("driver", driver)
    .load()

tableDF = [ID: bigint, CREATED_USER: string ... 9 more fields]


[ID: bigint, CREATED_USER: string ... 9 more fields]

In [23]:
tableDF.count()

361560

### Closing Spark Session

In [24]:
spark.stop()